In [2]:
import cv2
import numpy as np

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(8, (3,3), input_shape=(90,45,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(16,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

model.load_weights('a.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 88, 43, 8)         224       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 44, 21, 8)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 19, 16)        1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 21, 9, 16)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 7, 32)         4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 9, 3, 32)         0

In [4]:
# Load the Haar cascade classifiers
classifier_face = cv2.CascadeClassifier("./haar_cascade/lib/python3.10/site-packages/cv2/data/haarcascade_frontalface_default.xml")
classifier_smile = cv2.CascadeClassifier("./haar_cascade/lib/python3.10/site-packages/cv2/data/haarcascade_smile.xml")

In [5]:
# Initialize the webcam capture object
capture = cv2.VideoCapture(0)

# Loop until the user presses 'q' to quit
while True:
    # Read the next frame from the webcam
    ret, frame = capture.read()

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect objects using the first classifier
    objects1 = classifier_face.detectMultiScale(gray_frame)

    # Loop through the detected objects and draw a rectangle around them
    for (x, y, w, h) in objects1:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        get_face = frame[y:y+h, x:x+w]

    # Detect objects using the second classifier
    gray_face = cv2.cvtColor(get_face, cv2.COLOR_BGR2GRAY)
    objects2 = classifier_smile.detectMultiScale(gray_face)

    # Loop through the detected objects and draw a rectangle around them
    # for (x, y, w, h) in objects2:
    #     if objects2 is None or y < objects2[1]:
    #         selected_object = (x, y, w, h)
    #         cv2.rectangle(get_face, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
    # Select the object of interest based on its lower middle position in the image
    selected_object = None
    for (x, y, w, h) in objects2:
        cx = x + w // 2
        cy = y + h
        if selected_object is None or cy > selected_object[1]:
            selected_object = (cx, cy, w, h)

    # Draw a rectangle around the selected object
    if selected_object is not None:
        cx, cy, w, h = selected_object
        cv2.rectangle(get_face, (cx-w//2, cy-h), (cx+w//2, cy), (0, 255, 0), 2)
        object_image = get_face[cy-h:cy, cx-w//2:cx+w//2]
  
    smile = cv2.resize(object_image, (45,90))

    y_pred = model.predict(np.asarray([smile]))
    y_pred = (y_pred>0.5)*1
    print(y_pred)

    # Show the frame with the detected objects
    cv2.imshow('Objects', frame)

    # Check if the user pressed 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam capture object
capture.release()

NameError: name 'get_face' is not defined

In [11]:
# Release the webcam capture object
capture.release()